In [3]:
import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np
import os

In [14]:
current_dir = os.getcwd()

In [22]:

label_files = os.path.join(current_dir, 'Label_Files_2')
vector_files = os.path.join(current_dir, 'Vector_files_2')

label_list = []
vector_list =[]


for filename in os.listdir(label_files):
    try:
        file_path = os.path.join(label_files, filename)
        label_list.append(torch.load(file_path).detach())
    except:
        print(filename)

for filename in os.listdir(vector_files):
    file_path = os.path.join(vector_files, filename)
    vector_list.append(torch.load(file_path).detach())

.ipynb_checkpoints


In [23]:
labels = torch.cat(label_list).float()

# Divide all the labels by 4 
labels /= 4

#labels -=.000001

final_pool = torch.cat(vector_list).detach()

scaler = StandardScaler()

# Fit the scaler to your data
scaler.fit(final_pool)

# Transform your data using the scaler
normalized_values = scaler.transform(final_pool)

In [32]:
# convert our data into a torch dataset
#need y data from the file
ratings = torch.tensor(labels).float()
print(labels[5])
# Add a validation set 
x_train, x_test, y_train, y_test = train_test_split(final_pool, ratings, test_size=0.15)

# Data processing
data = torch.utils.data.TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 6, shuffle = True)

tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 

/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_31057/4040949071.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ratings = torch.tensor(labels).float()
/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_31057/4040949071.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.utils.data.TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())


In [25]:
model = torch.nn.Sequential(   
    torch.nn.Linear(768,64),
    torch.nn.SiLU(),
    torch.nn.Linear(64,16),
    torch.nn.SiLU(),
    torch.nn.Linear(16,1)
)
# Figure out whether its coming from sigmoid, loss function, or something else 

In [26]:
scoring_function = torch.nn.BCEWithLogitsLoss(reduce=False)#, reduction='none')

num_epochs = 20

optimizer = torch.optim.Adam( model.parameters(), .01 )

/Users/kaijavanzante/opt/anaconda3/envs/py4sci/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [27]:
def train():
    
    for epoch in range(num_epochs):
        
        epoch_losses = []
        
        for batch in loader:
            
            optimizer.zero_grad()
            
            # prepare the input and output tensors for the current batch
            batchX = torch.tensor(batch[0], dtype=torch.float32)
            batchY = torch.tensor(batch[1], dtype=torch.float32)
            batchY = batchY.unsqueeze(1)
            # forward pass
            y_pred = model.forward(batchX)
            print(y_pred)
            

            # Compute the loss
            loss = scoring_function(y_pred, batchY )
            #print(loss.shape)
            epoch_losses.append(loss)
            
            # Compute the gradient of the error with respect to the model parameters
            
            
            loss.mean().backward()
        
            # update the model parameters
            optimizer.step()
        
        # print the epoch and the average error for the epoch
        all_epoch_loss = torch.cat(epoch_losses)
        
        print('Epoch {}, Average Error: {}'.format(epoch, all_epoch_loss.mean()))
        
            

In [28]:
train()

tensor([[-0.1088],
        [-0.0994],
        [-0.1012],
        [-0.1018],
        [-0.1017],
        [-0.1027]], grad_fn=<AddmmBackward0>)


/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_31057/2784075520.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batchX = torch.tensor(batch[0], dtype=torch.float32)
/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_31057/2784075520.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batchY = torch.tensor(batch[1], dtype=torch.float32)


ValueError: Target size (torch.Size([6, 1, 512])) must be the same as input size (torch.Size([6, 1]))